In [3]:
import os
from datetime import datetime
import json

# Define the path to the ohlcvs_bybit directory
base_path = "E://git/passivbot/historical_data/ohlcvs_bybit"

# Define the cutoff date
start_date = "2024-01-01"
cutoff_date = datetime.strptime(start_date, "%Y-%m-%d")

# Create a dictionary to store coins available on or before the cutoff date
eligible_coins = {}

for coin in os.listdir(base_path):
    coin_path = os.path.join(base_path, coin)
    if not os.path.isdir(coin_path):
        continue
    dates = []
    for filename in os.listdir(coin_path):
        if filename.endswith(".npy"):
            try:
                date_str = filename.replace(".npy", "")
                date_obj = datetime.strptime(date_str, "%Y-%m-%d")
                dates.append(date_obj)
            except ValueError:
                continue
    if dates:
        earliest = min(dates)
        if earliest <= cutoff_date:
            # Use the earliest date across all sources
            if coin not in eligible_coins or earliest <= datetime.strptime(eligible_coins[coin], "%Y-%m-%d"):
                eligible_coins[coin] = earliest.strftime("%Y-%m-%d")

# Sort by earliest date
eligible_coins_sorted = dict(sorted(eligible_coins.items(), key=lambda x: x[1]))

orig_list_name = 'top_50_coins_300_mcap_5_usd_bybit'
with open(f"../configs/{orig_list_name}.json") as f1:
    orig_coins = json.load(f1)

coins = list(eligible_coins_sorted.keys())
coin_set = list(set(orig_coins) & set(coins))

json.dump(coin_set, open(f'../configs/{orig_list_name}_{start_date.replace("-", "")}.json', "w"))
